In [1]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
IMG_SIZE = 224
TURKISH_IMG_PATH = "Turkish"
TURKISH_LABEL_PATH = "labels_turkish"
CHINESE_IMG_PATH = "Chinese"
CHINESE_LABEL_PATH = "labels_chinese"

In [4]:
import shutil

def load_dataset(image_folder, label_folder, subclass_label, blur_prob=0.3, distort_prob=0.3, augment_copies=2, debug=False):
    X, y_drone, y_subclass = [], [], []

    for fname in tqdm(os.listdir(image_folder)):
        if not fname.lower().endswith(('.jpg', '.png')):
            continue

        image_path = os.path.join(image_folder, fname)
        label_path = os.path.join(label_folder, os.path.splitext(fname)[0] + ".txt")

        img_original = cv2.imread(image_path)
        if img_original is None:
            continue
        img_original = cv2.resize(img_original, (IMG_SIZE, IMG_SIZE))

        # ========== Process Original Image ==========
        img = img_original.astype("float32") / 255.0
        label = 0
        subclass = 0 if subclass_label == "turkish" else 1

        if not os.path.exists(label_path):
            label = 1
            subclass = 2
        else:
            with open(label_path, 'r') as f:
                if len(f.readlines()) == 0:
                    label = 1
                    subclass = 2

        X.append(img)
        y_drone.append(label)
        y_subclass.append(subclass)

        # ========== Augment and Save Copies ==========
        base_name = os.path.splitext(fname)[0]
        ext = os.path.splitext(fname)[1]

        for i in range(augment_copies):
            aug_img = img_original.copy()

            # Apply blur
            if random.random() < blur_prob:
                aug_img = cv2.GaussianBlur(aug_img, (5, 5), 0)

            # Apply brightness/contrast
            if random.random() < distort_prob:
                alpha = 1.0 + np.random.uniform(-0.3, 0.3)
                beta = np.random.randint(-30, 30)
                aug_img = cv2.convertScaleAbs(aug_img, alpha=alpha, beta=beta)

            aug_img = cv2.resize(aug_img, (IMG_SIZE, IMG_SIZE))
            aug_img_float = aug_img.astype("float32") / 255.0

            # Save augmented image
            aug_fname = f"{base_name}_aug{i}{ext}"
            aug_img_path = os.path.join(image_folder, aug_fname)
            cv2.imwrite(aug_img_path, aug_img)

            # Copy label file with same content
            if os.path.exists(label_path):
                aug_label_path = os.path.join(label_folder, f"{base_name}_aug{i}.txt")
                shutil.copyfile(label_path, aug_label_path)

            # Store in arrays
            X.append(aug_img_float)
            y_drone.append(label)
            y_subclass.append(subclass)

            if debug:
                plt.imshow(cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB))
                plt.title(f"Augmented Saved: {aug_fname}")
                plt.axis("off")
                plt.show()

    return np.array(X), np.array(y_drone), np.array(y_subclass)


In [6]:
X_turkish, y_drone_turkish, y_subclass_turkish = load_dataset(TURKISH_IMG_PATH, TURKISH_LABEL_PATH, "turkish")
X_chinese, y_drone_chinese, y_subclass_chinese = load_dataset(CHINESE_IMG_PATH, CHINESE_LABEL_PATH, "chinese")

X = np.concatenate([X_turkish, X_chinese])
y_drone = np.concatenate([y_drone_turkish, y_drone_chinese])
y_subclass = np.concatenate([y_subclass_turkish, y_subclass_chinese])

100%|██████████| 1272/1272 [00:17<00:00, 71.35it/s]


In [7]:
y_subclass = np.where(y_subclass == 2, 0, y_subclass)
y_subclass_cat = to_categorical(y_subclass, num_classes=2)

In [8]:
X_train, X_val, y_drone_train, y_drone_val, y_sub_train, y_sub_val = train_test_split(
    X, y_drone, y_subclass_cat, test_size=0.2, stratify=y_drone, random_state=42
)

In [9]:
inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

drone_output = Dense(1, activation='sigmoid', name='drone_output')(x)
subclass_output = Dense(2, activation='softmax', name='subclass_output')(x)

model = Model(inputs=inputs, outputs=[drone_output, subclass_output])
model.compile(
    optimizer=Adam(1e-4),
    loss={'drone_output': 'binary_crossentropy', 'subclass_output': 'categorical_crossentropy'},
    metrics={'drone_output': 'accuracy', 'subclass_output': 'accuracy'}
)

In [ ]:
history = model.fit(
    X_train,
    {'drone_output': y_drone_train, 'subclass_output': y_sub_train},
    validation_data=(X_val, {'drone_output': y_drone_val, 'subclass_output': y_sub_val}),
    epochs=25,
    batch_size=16
)

model.save("drone_classifier_Aug.h5")
print("Model saved")

Epoch 1/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 45s 133ms/step - drone_output_accuracy: 0.9880 - drone_output_loss: 0.0745 - loss: 0.7551 - subclass_output_accuracy: 0.5962 - subclass_output_loss: 0.6806 - val_drone_output_accuracy: 0.9946 - val_drone_output_loss: 0.0223 - val_loss: 0.5572 - val_subclass_output_accuracy: 0.7440 - val_subclass_output_loss: 0.5351
Epoch 2/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 42s 129ms/step - drone_output_accuracy: 0.9962 - drone_output_loss: 0.0218 - loss: 0.5038 - subclass_output_accuracy: 0.7762 - subclass_output_loss: 0.4820 - val_drone_output_accuracy: 0.9984 - val_drone_output_loss: 0.0174 - val_loss: 0.4444 - val_subclass_output_accuracy: 0.8084 - val_subclass_output_loss: 0.4272
Epoch 3/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 42s 129ms/step - drone_output_accuracy: 0.9968 - drone_output_loss: 0.0093 - loss: 0.3869 - subclass_output_accuracy: 0.8334 - subclass_output_loss: 0.3776 - val_drone_output_accuracy: 0.9984 - val_drone_output_loss: 0.0035 - val_loss: 0.3619 - v

In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2

IMG_SIZE = 224  # Use same size as training

model = load_model("drone_classifier_Aug.h5")

In [2]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # shape: (1, 224, 224, 3)
    return img


In [3]:
def predict_image(image_path):
    img = preprocess_image(image_path)

    # Get both outputs
    preds = model.predict(img)
    drone_pred = preds[0][0][0]  # output from 'drone_output'
    subclass_probs = preds[1][0]  # output from 'subclass_output'

    if drone_pred < 0.5:
        print(f"🛑 Not a Drone (score: {drone_pred:.2f})")
    else:
        subclass = np.argmax(subclass_probs)
        subclass_label = "Turkish" if subclass == 0 else "Chinese"
        print(f"✅ Drone Detected (score: {drone_pred:.2f}) — Subclass: {subclass_label} (conf: {subclass_probs[subclass]:.2f})")


In [7]:
predict_image("Turkish\\turkish_0199_aug0_aug0.jpg")
#predict_image("Chinese/sample_china.jpg")
#predict_image("non_drones/random_object.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
🛑 Not a Drone (score: 0.00)
